# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233570063856                   -0.50    8.0         
  2   -7.249994390960       -1.78       -1.39    1.0   7.81ms
  3   -7.250926343584       -3.03       -1.86    2.0   8.84ms
  4   -7.250999837656       -4.13       -1.89    2.0   9.36ms
  5   -7.251328381259       -3.48       -2.61    1.0   7.94ms
  6   -7.251337583201       -5.04       -3.05    1.0   7.84ms
  7   -7.251338699756       -5.95       -3.73    2.0   9.33ms
  8   -7.251338784640       -7.07       -3.99    3.0   11.7ms
  9   -7.251338797762       -7.88       -4.69    1.0   8.23ms
 10   -7.251338798629       -9.06       -4.99    3.0   11.6ms
 11   -7.251338798692      -10.20       -5.37    1.0   8.45ms
 12   -7.251338798701      -11.08       -5.71    1.0   8.64ms
 13   -7.251338798704      -11.48       -6.32    2.0   9.94ms
 14   -7.251338798704      -12.31       -6.57    3.0   11.6ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05671973824637365
[ Info: Arnoldi iteration step 2: normres = 0.6360913422097373
[ Info: Arnoldi iteration step 3: normres = 0.7895865798999196
[ Info: Arnoldi iteration step 4: normres = 0.2625695222587917
[ Info: Arnoldi iteration step 5: normres = 0.4623644037454543
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.04e-02, 7.19e-02, 3.90e-01, 2.36e-01, 1.91e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3174942742295876
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.78e-03, 1.91e-01, 2.16e-01, 1.10e-01, 4.04e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07616536492619268
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.16e-04, 1.49e-02, 1.06e-02, 3.69e-02, 5.65e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1194086649592165
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.11e-05